In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
X_train = pd.read_csv('../data/X_train_flu.csv', index_col= 'respondent_id')
X_test = pd.read_csv('../data/X_test_flu.csv', index_col= 'respondent_id')
y_train = pd.read_csv('../data/y_train_flu.csv', index_col= 'respondent_id')
y_test = pd.read_csv('../data/y_test_flu.csv', index_col= 'respondent_id')

In [3]:
y_train = y_train['seasonal_vaccine']
y_test = y_test['seasonal_vaccine']

In [4]:
X_train_cat = X_train.select_dtypes('object')
X_train_num = X_train.select_dtypes(['float64', 'int64'])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])
num_pipe = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
])

transformer = ColumnTransformer([('categorical', cat_pipe, X_train_cat.columns),
                                 ('numerical', num_pipe, X_train_num.columns)])

# XGBoost

In [5]:
model_pipe = Pipeline(steps=[
    ('trans', transformer),
    ('xgboost', XGBClassifier(n_jobs=-1, random_state=42))
])

In [6]:
cv_XGBoost1 = cross_val_score(model_pipe, X_train, y_train, n_jobs=-1, verbose=2, scoring='f1')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    8.2s remaining:   12.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.3s finished


In [7]:
cv_XGBoost1.mean()

0.7465664433767254

# Grid Search on XGBoost

In [8]:
model_pipe_2 = Pipeline(steps=[
    ('trans', transformer),
    ('xgboost', XGBClassifier(n_jobs=-1, random_state=42))
])

In [9]:
params = {
        'xgboost__min_child_weight': [1, 5, 10],
        'xgboost__gamma': [0.5, 1, 1.5, 2, 5],
        'xgboost__subsample': [0.6, 0.8, 1.0],
        'xgboost__colsample_bytree': [0.6, 0.8, 1.0],
        'xgboost__max_depth': [3, 4, 5]
        }

In [ ]:
gs_XGBoost = GridSearchCV(model_pipe_2, param_grid=params, n_jobs=-1, verbose=2, return_train_score=True, scoring='f1')

In [ ]:
gs_XGBoost.fit(X_train, y_train)

In [ ]:
gs_XGBoost.best_params_

In [ ]:
cv_XGBoost = cross_val_score(gs_XGBoost.best_estimator, X_train, y_train, n_jobs=-1, verbose=3, scoring='f1')

In [ ]:
cv_XGBoost.mean()